<a href="https://colab.research.google.com/github/deb-david/mlproject/blob/main/assessment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 1 for ITO5212 - TP5

**Student Name:**  Deborrah David

**Student ID:**    23026952
## Part 1:  Text Classification

General comments and any shared processing here.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Install necessary packages
!pip install numpy pandas nltk scikit-learn matplotlib tensorflow keras

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [3]:
# Download required nltk resources
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
# Install necessary packages, including torchtext
!pip install torch torchtext


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 20.0 MB/s eta 0:00:00


In [4]:
# Import libraries
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer, WordNetLemmatizer

# Import scikit-learn utilities for text processing and modeling
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (f1_score, precision_score, recall_score,
                             precision_recall_curve, confusion_matrix)
from sklearn.model_selection import cross_val_score, train_test_split

# Import matplotlib for visualization
import matplotlib.pyplot as plt

# Import TensorFlow and Keras for deep learning
import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding, SpatialDropout1D


In [5]:
# Import PyTorch
import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
!pip uninstall torch torchtext -y

Found existing installation: torch 2.4.0+cu121
Uninstalling torch-2.4.0+cu121:
  Successfully uninstalled torch-2.4.0+cu121


In [ ]:
# Install compatible versions of PyTorch, torchtext, torchvision, and torchaudio
!pip install torch torchvision torchaudio torchtext --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 10.9 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchtext to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 47.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 47.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 41.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 48.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 46.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 49.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 40.7 MB/s eta 0:00:00
INFO: pip is still looking at multiple versions of torchtext to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━

In [6]:
import torch
import torchtext
from torchtext.data import get_tokenizer # updated import statement

# example of how to use get_tokenizer
tokenizer = get_tokenizer("basic_english")
tokens = tokenizer("You can now use torchtext.data.get_tokenizer")
print(tokens)

['you', 'can', 'now', 'use', 'torchtext', '.', 'data', '.', 'get_tokenizer']


### **Load and Preprocess the Data**

In [7]:
df_train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/ITO5212 Data Analytics for semi-structured data/train.csv", delimiter='\t', header=None, names=['sentence_source', 'label', 'label_notes', 'sentence'])
df_test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/ITO5212 Data Analytics for semi-structured data/test.csv", delimiter='\t', header=None, names=['sentence_source', 'label', 'label_notes', 'sentence'])

In [8]:
# Split df_train into a smaller training set and a validation set
df_train, df_dev = train_test_split(df_train, test_size=0.05, random_state=42)
# 1% for validation

In [9]:
print(df_dev.head(10))  # Displays the first 5 rows by default

                                           sentence_source  label  \
775529   How can you not love something from your child...    NaN   
156395   I am normally not interested in stories about ...    NaN   
542553   well, compare this movie to 'notebook' is simp...    NaN   
477058   It won't take very long to realize that Suppor...    NaN   
1023410  I loved it. I bought books at the same time, s...    NaN   
863955   Having read some of the reviews of the directo...    NaN   
510719   I used to purchase filters in the lower price ...    NaN   
1399825  The buttons are a little sticky, but they work...    NaN   
932859   The Author has had a lot to forgive in her lif...    NaN   
147349   The Return of the Dragon , although dated , in...    NaN   

         label_notes  sentence  
775529           NaN       NaN  
156395           NaN       NaN  
542553           NaN       NaN  
477058           NaN       NaN  
1023410          NaN       NaN  
863955           NaN       NaN  
510719   

In [10]:
print(df_dev.info())

<class 'pandas.core.frame.DataFrame'>
Index: 79957 entries, 775529 to 823120
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   sentence_source  79957 non-null  object 
 1   label            0 non-null      float64
 2   label_notes      0 non-null      float64
 3   sentence         0 non-null      float64
dtypes: float64(3), object(1)
memory usage: 3.1+ MB
None


In [11]:
print(df_dev.describe())

       label  label_notes  sentence
count    0.0          0.0       0.0
mean     NaN          NaN       NaN
std      NaN          NaN       NaN
min      NaN          NaN       NaN
25%      NaN          NaN       NaN
50%      NaN          NaN       NaN
75%      NaN          NaN       NaN
max      NaN          NaN       NaN


### **Text Preprocessing with Stopwords and Stemming:**

In [12]:
print(f'Number of training examples: {len(df_train)}')
print(f'Number of validation examples: {len(df_dev)}')
print(f'Number of testing examples: {len(df_test)}')

Number of training examples: 1519181
Number of validation examples: 79957
Number of testing examples: 399786


In [26]:
# Assuming 'en_core_web_sm' is the desired spaCy model
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 51.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [13]:
# Extract unique labels from the dev data
unique_labels = set(row['label'] for _, row in df_dev.iterrows()) # Iterate over rows and access 'label' column
print("Unique Labels:", unique_labels)

Unique Labels: {nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,

In [14]:
# Extract unique labels from the 'label' column in the DataFrame
unique_labels = set(df_dev['label'])
print("Unique Labels:", unique_labels)

Unique Labels: {nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,

In [15]:
from torchtext.vocab import build_vocab_from_iterator
from collections import Counter

# You'll need to define a function to yield tokens from your dataset
def yield_tokens(data_iter):
    for _, row in data_iter:
        # Access the correct column name. It was 'sentence' not 'text'
        # Check if the value is a string before splitting
        if isinstance(row['sentence'], str):
            yield row['sentence'].split()
        else:
            # Handle non-string values (e.g., NaN) - you might want to skip or replace them
            yield [] # or yield ["<unk>"]

# Build the vocabulary from your training data
# Create a counter first
counter = Counter()
for tokens in yield_tokens(df_train.iterrows()):
    counter.update(tokens)

# Build vocab from counter and specify special tokens
vocab = build_vocab_from_iterator(yield_tokens(df_train.iterrows()))
# The specials argument has been removed.
vocab.set_default_index(vocab["<unk>"])


# Now you can access the vocabulary
print("Vocabulary Sample:", list(vocab.itos)[:10])

# Extract unique labels from the 'label' column in the DataFrame
unique_labels = set(df_dev['label'])
print("Unique Labels:", unique_labels)


vocab_size = len(vocab.itos)
print(f"Unique tokens in TEXT vocabulary: {vocab_size}")
print(f"Unique tokens in LABEL vocabulary: {len(unique_labels)}")

1519181lines [01:49, 13853.50lines/s]


AttributeError: 'Vocab' object has no attribute 'set_default_index'

Text Vocabulary Size: The text vocabulary is very large, which is typical when working with a large dataset and/or a very diverse set of tokens.

Label Vocabulary Size: The label vocabulary is also quite large. This is due to a high number of unique labels or errors in the labeling process.



In [16]:
print(df_dev.columns)

Index(['sentence_source', 'label', 'label_notes', 'sentence'], dtype='object')


In [17]:
print(df_dev.info())

<class 'pandas.core.frame.DataFrame'>
Index: 79957 entries, 775529 to 823120
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   sentence_source  79957 non-null  object 
 1   label            0 non-null      float64
 2   label_notes      0 non-null      float64
 3   sentence         0 non-null      float64
dtypes: float64(3), object(1)
memory usage: 3.1+ MB
None


In [18]:
# Display the distribution of labels
print(df_dev['label'].value_counts())


Series([], Name: count, dtype: int64)


In [19]:
# Display the first few entries of label_notes
print(df_dev['label_notes'].head())


775529    NaN
156395    NaN
542553    NaN
477058    NaN
1023410   NaN
Name: label_notes, dtype: float64


In [ ]:
# Display the first few sentences
print(df_dev['sentence'].head())


775529    NaN
156395    NaN
542553    NaN
477058    NaN
1023410   NaN
Name: sentence, dtype: float64


In [ ]:
# Check for missing values in the DataFrame
print(df_dev.isnull().sum())


sentence_source       0
label              7996
label_notes        7996
sentence           7996
dtype: int64


In [ ]:
# Display a random sample of the DataFrame
print(df_dev.sample(10))

                                           sentence_source  label  \
748408   I got this book from "kindle daily deals" befo...    NaN   
255278   Put about 10 hours on this drive so far so goo...    NaN   
621964   This is a page turner as far as I am concerned...    NaN   
1409146  This is a well made documentary about a rare h...    NaN   
1473946  Make sure you get the Synapse blu-ray / DVD co...    NaN   
1250967  I bought these as I have swimmer's ear and don...    NaN   
1518808  I liked pretty much everything about it. Story...    NaN   
383610   I am trying to get my daughter to use her Olym...    NaN   
806523   New Amsterdam is set in an alternate fantastic...    NaN   
880109   This was a high quality connector with gold pl...    NaN   

         label_notes  sentence  
748408           NaN       NaN  
255278           NaN       NaN  
621964           NaN       NaN  
1409146          NaN       NaN  
1473946          NaN       NaN  
1250967          NaN       NaN  
1518808  

In [ ]:
import pandas as pd

# If you have the labels in a DataFrame, you can check for inconsistencies
label_counts = df_dev['label'].value_counts()
print("Label Counts (Top 10):")
print(label_counts.head(10))

# Optionally, filter out rare labels or correct inconsistencies
# For example, removing labels that appear very infrequently
threshold = 10  # example threshold
common_labels = label_counts[label_counts >= threshold].index
# Use df_dev instead of df_train to filter based on the common labels in df_dev
filtered_df_dev = df_dev[df_dev['label'].isin(common_labels)]

Label Counts (Top 10):
Series([], Name: count, dtype: int64)


In [ ]:
# Display the first few rows of the DataFrame
print(df_dev.head())

# Display the column names
print(df_dev.columns)


                                           sentence_source  label  \
775529   How can you not love something from your child...    NaN   
156395   I am normally not interested in stories about ...    NaN   
542553   well, compare this movie to 'notebook' is simp...    NaN   
477058   It won't take very long to realize that Suppor...    NaN   
1023410  I loved it. I bought books at the same time, s...    NaN   

         label_notes  sentence  
775529           NaN       NaN  
156395           NaN       NaN  
542553           NaN       NaN  
477058           NaN       NaN  
1023410          NaN       NaN  
Index(['sentence_source', 'label', 'label_notes', 'sentence'], dtype='object')


In [ ]:
# Check for non-empty values in the 'label' column
print(df_dev['label'].unique())


[nan]


In [ ]:
# Display data types of each column
print(df_dev.dtypes)


sentence_source     object
label              float64
label_notes        float64
sentence           float64
dtype: object


In [ ]:
# Test value counts on a sample column
test_series = pd.Series(['a', 'b', 'a', 'c', 'b', 'b'])
print(test_series.value_counts())


b    3
a    2
c    1
Name: count, dtype: int64


### **Pre-processing Data**

In [23]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import string

nltk.download('punkt')
nltk.download('stopwords')

def preprocess_text(text):
    # Lowercasing
    text = text.lower()
    # Tokenization
    tokens = word_tokenize(text)
    # Removing punctuation and numbers
    tokens = [word for word in tokens if word.isalpha()]
    # Removing stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    # Stemming
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(word) for word in tokens]
    return ' '.join(tokens)

# Apply preprocessing to the 'sentence_source' column

df_dev['cleaned_text'] = df_dev['sentence_source'].apply(preprocess_text)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [30]:
def preprocess_text(text):
    # Lowercasing
    text = text.lower()
    # Tokenization
    tokens = word_tokenize(text)
    # Removing punctuation and numbers
    tokens = [word for word in tokens if word.isalpha()]
    # Removing stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    # Stemming
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(word) for word in tokens]
    return ' '.join(tokens)

# Apply preprocessing to the 'sentence_source' column

df_test['cleaned_text'] = df_test['sentence_source'].apply(preprocess_text)

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import string

nltk.download('punkt')
nltk.download('stopwords')

def preprocess_text(text):
    # Lowercasing
    text = text.lower()
    # Tokenization
    tokens = word_tokenize(text)
    # Removing punctuation and numbers
    tokens = [word for word in tokens if word.isalpha()]
    # Removing stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    # Stemming
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(word) for word in tokens]
    return ' '.join(tokens)

# Apply preprocessing to the 'sentence_source' column

df_train['cleaned_text'] = df_train['sentence_source'].apply(preprocess_text)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [29]:
print(df_train[['cleaned_text']].head())


KeyError: "None of [Index(['cleaned_text'], dtype='object')] are in the [columns]"

In [ ]:
print(df_dev[['sentence_source', 'cleaned_text']].head())


                                           sentence_source  \
775529   How can you not love something from your child...   
156395   I am normally not interested in stories about ...   
542553   well, compare this movie to 'notebook' is simp...   
477058   It won't take very long to realize that Suppor...   
1023410  I loved it. I bought books at the same time, s...   

                                              cleaned_text  
775529   love someth childhood use beg stay watch bewit...  
156395   normal interest stori spirit depart find convi...  
542553   well compar movi simpli much one worst movi ev...  
477058   wo take long realiz support local sheriff litt...  
1023410  love bought book time read event happen point ...  


In [ ]:
# some data samples belong to class 0
# Check if any rows meet the condition before sampling
if (df_dev.label == 0).any():
  df_dev.loc[df_dev.label == 0].sample(5)[['sentence', 'label']]
else:
  print("No data samples with label 0 found.")

No data samples with label 0 found.


In [ ]:
# Note that you just need to downoad punk once in your machine, but ev time o Colab as i deletes everything when the connection is terminated
import nltk
nltk.download('punkt')

data_file_path = '/content/drive/MyDrive/ITO5212 Data Analytics for semi-structured data/Assessment 2/train.csv'

#importing the data
data_CBOW = CBOWDataSet(data_file_path=data_file_path)

#### **Word2Vce & TF-IDF**

In [25]:
# Prepare TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=10000)
# Fit the vectorizer to the dev data
tfidf_vectorizer.fit(df_dev['cleaned_text']) # Fit the vectorizer to learn vocabulary and idf
X_dev_tfidf = tfidf_vectorizer.transform(df_dev['cleaned_text'])


In [26]:
# Word2Vec Training
from gensim.models import Word2Vec # import the Word2Vec class

sentences = df_dev['cleaned_text'].apply(str.split).tolist()
word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=2, workers=4)
word2vec_model.train(sentences, total_examples=len(sentences), epochs=10)

# Transform the Word2Vec embeddings to use as features
def get_word2vec_embeddings(text, model, vector_size):
    tokens = text.split()
    vec = np.zeros(vector_size)
    for word in tokens:
        if word in model.wv:
            vec += model.wv[word]
    return vec / len(tokens) if len(tokens) > 0 else vec

X_dev_w2v = np.array([get_word2vec_embeddings(text, word2vec_model, 100) for text in df_dev['cleaned_text']])

In [27]:
# Prepare TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=10000)
# Fit the vectorizer to the test data
tfidf_vectorizer.fit(df_test['cleaned_text']) # Fit the vectorizer to learn vocabulary and idf
X_test_tfidf = tfidf_vectorizer.transform(df_test['cleaned_text'])


KeyError: 'cleaned_text'

In [ ]:
# Word2Vec Training
from gensim.models import Word2Vec # import the Word2Vec class

sentences = df_dev['cleaned_text'].apply(str.split).tolist()
word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=2, workers=4)
word2vec_model.train(sentences, total_examples=len(sentences), epochs=10)

# Transform the Word2Vec embeddings to use as features
def get_word2vec_embeddings(text, model, vector_size):
    tokens = text.split()
    vec = np.zeros(vector_size)
    for word in tokens:
        if word in model.wv:
            vec += model.wv[word]
    return vec / len(tokens) if len(tokens) > 0 else vec

X_test_w2v = np.array([get_word2vec_embeddings(text, word2vec_model, 100) for text in df_test['cleaned_text']])

In [ ]:
# Prepare TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=10000)
# Fit the vectorizer to the test data
# Check for typos and ensure that "cleaned_text" is the correct column name
tfidf_vectorizer.fit(df_train['cleaned_text']) # Fit the vectorizer to learn vocabulary and idf
X_train_tfidf = tfidf_vectorizer.transform(df_train['cleaned_text'])


In [ ]:
# Word2Vec Training
sentences = df_dev['cleaned_text'].apply(str.split).tolist()
word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=2, workers=4)
word2vec_model.train(sentences, total_examples=len(sentences), epochs=10)

# Transform the Word2Vec embeddings to use as features
def get_word2vec_embeddings(text, model, vector_size):
    tokens = text.split()
    vec = np.zeros(vector_size)
    for word in tokens:
        if word in model.wv:
            vec += model.wv[word]
    return vec / len(tokens) if len(tokens) > 0 else vec

X_train_w2v = np.array([get_word2vec_embeddings(text, word2vec_model, 100) for text in df_train['cleaned_text']])

# Print the columns of df_train to verify the existence of 'cleaned_text'
print(df_train.columns)

Index(['sentence_source', 'label', 'label_notes', 'sentence', 'cleaned_text'], dtype='object')


In [ ]:
# Prepare TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=10000)
# Fit the vectorizer to the test data
tfidf_vectorizer.fit(df_dev['cleaned_text']) # Fit the vectorizer to learn vocabulary and idf
X_train_tfidf = tfidf_vectorizer.transform(df_train['cleaned_text'])

# Word2Vec Training
sentences = df_dev['cleaned_text'].apply(str.split).tolist()
word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=2, workers=4)
word2vec_model.train(sentences, total_examples=len(sentences), epochs=10)

# Transform the Word2Vec embeddings to use as features
def get_word2vec_embeddings(text, model, vector_size):
    tokens = text.split()
    vec = np.zeros(vector_size)
    for word in tokens:
        if word in model.wv:
            vec += model.wv[word]
    return vec / len(tokens) if len(tokens) > 0 else vec

X_train_w2v = np.array([get_word2vec_embeddings(text, word2vec_model, 100) for text in df_train['cleaned_text']])


### **Visualising Word Embeddings (Word2Vec)**

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

# Visualize Word2Vec embeddings using t-SNE
def visualize_word_embeddings(word2vec_model):
    words = list(word2vec_model.wv.index_to_key)[:200]  # Select 200 words for visualization
    word_vectors = [word2vec_model.wv[word] for word in words]

    tsne = TSNE(n_components=2, random_state=42)
    word_vectors_tsne = tsne.fit_transform(word_vectors)

    plt.figure(figsize=(10, 10))
    plt.scatter(word_vectors_tsne[:, 0], word_vectors_tsne[:, 1], edgecolors='k', c='r')

    for i, word in enumerate(words):
        plt.text(word_vectors_tsne[i, 0], word_vectors_tsne[i, 1], word)
    plt.title("Word Embeddings Visualization with t-SNE")
    plt.show()

visualize_word_embeddings(word2vec_model)


### **Identifying and Handling Data Imbalance**

In [ ]:
from imblearn.over_sampling import SMOTE
from collections import Counter

# Check label imbalance in df_train
print(f"Label distribution in df_train: {Counter(df_train['label'])}")
print(f"Label distribution in df_dev: {Counter(df_dev['label'])}")

# Handle imbalance using SMOTE for df_train
smote = SMOTE(random_state=42)
X_train_tfidf_balanced, y_train_balanced = smote.fit_resample(X_train_tfidf, df_train['label'])
print(f"Balanced label distribution in df_train: {Counter(y_train_balanced)}")


### Part 1A: Statistical Method

Details of method



In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import torch
from torch import nn

# Logistic Regression Model (TF-IDF features)
lr_model = LogisticRegression()
lr_model.fit(X_train_tfidf_balanced, y_train_balanced)
lr_dev_preds = lr_model.predict(X_dev_tfidf)

# Random Forest Model (TF-IDF features)
rf_model = RandomForestClassifier(n_estimators=100)
rf_model.fit(X_train_tfidf_balanced, y_train_balanced)
rf_dev_preds = rf_model.predict(X_dev_tfidf)

### Part 1B: RNN Method

Details of method


In [ ]:
# Simple RNN Model (Word2Vec features)
class SimpleRNN(nn.Module):
    def __init__(self, input_size):
        super(SimpleRNN, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size=128, num_layers=2, batch_first=True)
        self.fc = nn.Linear(128, 1)

    def forward(self, x):
        _, h_n = self.rnn(x)
        out = self.fc(h_n[-1])
        return torch.sigmoid(out).squeeze()

rnn_model = SimpleRNN(input_size=100)
optimizer = torch.optim.Adam(rnn_model.parameters(), lr=0.001)
criterion = nn.BCELoss()

# Training loop for RNN
def train_rnn_model(model, X_train, y_train, optimizer, criterion, epochs=5):
    model.train()
    for epoch in range(epochs):
        inputs = torch.tensor(X_train, dtype=torch.float32)
        labels = torch.tensor(y_train.values, dtype=torch.float32)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item()}")

train_rnn_model(rnn_model, X_train_w2v, df_train['label'], optimizer, criterion)

### Part 1C:  Results for Methods

F1, precision, etc.


In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, classification_report

# Logistic Regression Evaluation
lr_dev_preds = lr_model.predict(X_dev_tfidf)
lr_accuracy = accuracy_score(df_dev['label'], lr_dev_preds)
lr_f1 = f1_score(df_dev['label'], lr_dev_preds, average='weighted')
lr_precision = precision_score(df_dev['label'], lr_dev_preds, average='weighted')
lr_recall = recall_score(df_dev['label'], lr_dev_preds, average='weighted')

print("Logistic Regression Evaluation:")
print(f"Accuracy: {lr_accuracy}")
print(f"F1-Score: {lr_f1}")
print(f"Precision: {lr_precision}")
print(f"Recall: {lr_recall}")
print("\nConfusion Matrix:")
print(confusion_matrix(df_dev['label'], lr_dev_preds))

print("\nClassification Report:")
print(classification_report(df_dev['label'], lr_dev_preds))

# Random Forest Evaluation
rf_dev_preds = rf_model.predict(X_dev_tfidf)
rf_accuracy = accuracy_score(df_dev['label'], rf_dev_preds)
rf_f1 = f1_score(df_dev['label'], rf_dev_preds, average='weighted')
rf_precision = precision_score(df_dev['label'], rf_dev_preds, average='weighted')
rf_recall = recall_score(df_dev['label'], rf_dev_preds, average='weighted')

print("\nRandom Forest Evaluation:")
print(f"Accuracy: {rf_accuracy}")
print(f"F1-Score: {rf_f1}")
print(f"Precision: {rf_precision}")
print(f"Recall: {rf_recall}")
print("\nConfusion Matrix:")
print(confusion_matrix(df_dev['label'], rf_dev_preds))

print("\nClassification Report:")
print(classification_report(df_dev['label'], rf_dev_preds))

# SimpleRNN Evaluation
def evaluate_rnn(model, X_dev, y_dev):
    model.eval()
    with torch.no_grad():
        inputs = torch.tensor(X_dev, dtype=torch.float32)
        labels = torch.tensor(y_dev.values, dtype=torch.float32)
        outputs = model(inputs)
        preds = (outputs >= 0.5).int()

    accuracy = accuracy_score(y_dev, preds)
    f1 = f1_score(y_dev, preds, average='weighted')
    precision = precision_score(y_dev, preds, average='weighted')
    recall = recall_score(y_dev, preds, average='weighted')

    print("\nRNN Evaluation:")
    print(f"Accuracy: {accuracy}")
    print(f"F1-Score: {f1}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print("\nConfusion Matrix:")
    print(confusion_matrix(y_dev, preds))

    print("\nClassification Report:")
    print(classification_report(y_dev, preds))

# Evaluate the RNN model using Word2Vec embeddings
evaluate_rnn(rnn_model, X_dev_w2v, df_dev['label'])


### Part 1D: Plots for Methods

F1, precision, etc.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

# Function to plot confusion matrix
def plot_confusion_matrix(y_true, y_pred, title="Confusion Matrix"):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
    plt.title(title)
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()

# Logistic Regression Evaluation with Plots
lr_dev_preds = lr_model.predict(X_dev_tfidf)
lr_accuracy = accuracy_score(df_dev['label'], lr_dev_preds)
lr_f1 = f1_score(df_dev['label'], lr_dev_preds, average='weighted')
lr_precision = precision_score(df_dev['label'], lr_dev_preds, average='weighted')
lr_recall = recall_score(df_dev['label'], lr_dev_preds, average='weighted')

print("Logistic Regression Evaluation:")
print(f"Accuracy: {lr_accuracy}")
print(f"F1-Score: {lr_f1}")
print(f"Precision: {lr_precision}")
print(f"Recall: {lr_recall}")

# Plot Confusion Matrix for Logistic Regression
plot_confusion_matrix(df_dev['label'], lr_dev_preds, title="Logistic Regression Confusion Matrix")

# Random Forest Evaluation with Plots
rf_dev_preds = rf_model.predict(X_dev_tfidf)
rf_accuracy = accuracy_score(df_dev['label'], rf_dev_preds)
rf_f1 = f1_score(df_dev['label'], rf_dev_preds, average='weighted')
rf_precision = precision_score(df_dev['label'], rf_dev_preds, average='weighted')
rf_recall = recall_score(df_dev['label'], rf_dev_preds, average='weighted')

print("\nRandom Forest Evaluation:")
print(f"Accuracy: {rf_accuracy}")
print(f"F1-Score: {rf_f1}")
print(f"Precision: {rf_precision}")
print(f"Recall: {rf_recall}")

# Plot Confusion Matrix for Random Forest
plot_confusion_matrix(df_dev['label'], rf_dev_preds, title="Random Forest Confusion Matrix")

# SimpleRNN Evaluation with Plots
def evaluate_rnn(model, X_dev, y_dev):
    model.eval()
    with torch.no_grad():
        inputs = torch.tensor(X_dev, dtype=torch.float32)
        labels = torch.tensor(y_dev.values, dtype=torch.float32)
        outputs = model(inputs)
        preds = (outputs >= 0.5).int()

    accuracy = accuracy_score(y_dev, preds)
    f1 = f1_score(y_dev, preds, average='weighted')
    precision = precision_score(y_dev, preds, average='weighted')
    recall = recall_score(y_dev, preds, average='weighted')

    print("\nRNN Evaluation:")
    print(f"Accuracy: {accuracy}")
    print(f"F1-Score: {f1}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")

    # Plot Confusion Matrix for RNN
    plot_confusion_matrix(y_dev, preds, title="RNN Confusion Matrix")

# Evaluate the RNN model using Word2Vec embeddings
evaluate_rnn(rnn_model, X_dev_w2v, df_dev['label'])

# Visualizing Evaluation Metrics
def plot_metrics(model_name, accuracy, f1, precision, recall):
    metrics = [accuracy, f1, precision, recall]
    metric_names = ['Accuracy', 'F1-Score', 'Precision', 'Recall']

    plt.figure(figsize=(8, 5))
    plt.bar(metric_names, metrics, color=['skyblue', 'lightgreen', 'lightcoral', 'lightsalmon'])
    plt.ylim(0, 1)
    plt.title(f'{model_name} Metrics')
    plt.ylabel('Score')
    plt.show()

# Plot metrics for Logistic Regression
plot_metrics('Logistic Regression', lr_accuracy, lr_f1, lr_precision, lr_recall)

# Plot metrics for Random Forest
plot_metrics('Random Forest', rf_accuracy, rf_f1, rf_precision, rf_recall)


## Part 2: Topic Modelling

General comments and any shared processing here.
